In [9]:
!pip install -q transformers gradio

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 1.9 MB 46.4 MB/s 
     |████████████████████████████████| 206 kB 62.1 MB/s 
     |████████████████████████████████| 961 kB 53.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 3.5 MB 57.4 MB/s 


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained('twmkn9/bert-base-uncased-squad2')
model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/465 [00:00<?, ?B/s]

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/see--/bert-uncased-tf2-qa/1'.
INFO:absl:Downloading https://tfhub.dev/see--/bert-uncased-tf2-qa/1: 1.14GB
INFO:absl:Downloaded https://tfhub.dev/see--/bert-uncased-tf2-qa/1, Total size: 1.27GB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/see--/bert-uncased-tf2-qa/1'.


In [5]:
def qa_func(paragraph, question):
  question_tokens = tokenizer.tokenize(question)
  paragraph_tokens = tokenizer.tokenize(paragraph)
  tokens = ['[CLS]'] + question_tokens + ['[SEP]'] + paragraph_tokens + ['[SEP]']
  input_word_ids = tokenizer.convert_tokens_to_ids(tokens)
  input_mask = [1] * len(input_word_ids)
  input_type_ids = [0] * (1 + len(question_tokens) + 1) + [1] * (len(paragraph_tokens) + 1)

  input_word_ids, input_mask, input_type_ids = map(lambda t: tf.expand_dims(
      tf.convert_to_tensor(t, dtype=tf.int32), 0), (input_word_ids, input_mask, input_type_ids))
  outputs = model([input_word_ids, input_mask, input_type_ids])
  short_start = tf.argmax(outputs[0][0][1:]) + 1
  short_end = tf.argmax(outputs[1][0][1:]) + 1
  answer_tokens = tokens[short_start: short_end + 1]
  answer = tokenizer.convert_tokens_to_string(answer_tokens)
  return answer

In [6]:
paragraph = '''
Herald College Kathmandu (HCK) Herald College Kathmandu provides the best UK university education at most affordable fees. It offers Bachelor Degrees in IT and Business in direct partnership with the University of Wolverhampton, UK. Programmes that Herald College Kathmandu offers are flexible and have always aimed to foster a broader based knowledge.
Herald College Kathmandu offers the BSc (Hons) International Business Management and BSc (Hons) in Computer Science qualification from the University of Wolverhampton (WLV).
Considering effort, aptitude and knowledge, Herald College Kathmandu organizes a competition every year where students are awarded with scholarships to participate in 15-Day Summer-Camp organized at the University of Wolverhampton, UK.
'''

question = "What programs does Herald offer?"

In [13]:
import gradio as gr

iface = gr.Interface(qa_func, 
    [
        gr.inputs.Textbox(lines=7, label="Context", default=paragraph), 
        gr.inputs.Textbox(lines=1, label="Question", default=question), 
    ], 
    gr.outputs.Textbox(label="Answer"))

iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://28235.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://28235.gradio.app')